====== Nguồn http://users.soict.hust.edu.vn/khoattq/ml-dm-course/ ======



<h1><center>Tiền xử lý dữ liệu cho dữ liệu về  loại thuốc phản ứng của người bệnh</center></h1>

<h1>Nội dung thực hành</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#about_dataset">Giới thiệu về dữ liệu</a></li>
        <li><a href="#downloading_data">Tải dữ liệu</a></li>
        <li><a href="#pre-processing">Tiền xử lý dữ liệu</a></li>
    </ol>
</div>
<br>
<hr>

Import các thư viện sau:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>RandomForestClassifier</b> from <b>sklearn.ensemble</b> </li>
</ul>

In [2]:
import numpy as np
import pandas as pd

<div id="Giới thiệu về dữ liệu">
    <h2>Giới thiệu về dữ liệu</h2>
    Giả sử bạn là một người nghiên cứu về dữ liệu y học, bạn đã thu thập được một tập dữ liệu các bệnh nhân của cùng một loại bệnh, trong khi điều trị mỗi bệnh nhân phản ứng với 1 trong 5 loại thuốc A, B, C, X và Y.
    <br>
    <br>
    Công việc cần làm là xây dựng một mô hình nhằm tìm ra loại thuốc phù hợp cho bệnh nhân mới bị mắc loại bệnh trên. Các trường dữ liệu gồm gồm có: Age, Sex, Blood Pressur, Cholesterol, Na_to_K, Drug
    <br>
    <br>
    Chúng ta sử dụng phần dữ liệu huấn luyện để xây dựng mô hình Random Forest và đánh giá mô hình này trên tập đánh giá, sau đó sử dụng mô hình để phán đoán cho bệnh nhân mới
</div>


<div id="downloading_data"> 
    <h2>Load dữ liệu từ file</h2>
<!--     Sử dụng wget để tải dữ liệu từ IBM Object Storage. -->
</div>

Đọc dữ liệu vào pandas dataframe

Một cách đơn giản để lưu trữ các tập dữ liệu lớn là sử dụng tệp CSV (tệp được phân tách bằng dấu phẩy).

Đọc file: df = pd.read_csv('data.csv')

In toàn bộ data: print(df.to_string())

Trả về 5 hàng đầu và 5 hàng cuối: print(df) 

In [39]:
my_data = pd.read_csv("data/drug200.csv", delimiter=",")
my_data[0:10]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY
5,22,F,NORMAL,HIGH,8.607,drugX
6,49,F,NORMAL,HIGH,16.275,drugY
7,41,M,LOW,HIGH,11.037,drugC
8,60,M,NORMAL,HIGH,15.171,drugY
9,43,M,LOW,NORMAL,19.368,drugY


Kích thước của dữ liệu

In [40]:
my_data.shape

(200, 6)

<div href="pre-processing">
    <h2>Tiền xử lý dữ liệu</h2>
</div>

<b>my_data</b> là  Drug.csv sau khi đọc vào pandas dataframe, chúng ta định nghĩa các tham số sau: <br>

<ul>
    <li> <b> X </b> as the <b> Ma trận thuộc tính </b> (data of my_data) 
    <li> <b> y </b> as the <b> Vector thuốc phản hồi (target) </b> </li>
</ul>

In [41]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:10]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.114],
       [28, 'F', 'NORMAL', 'HIGH', 7.798],
       [61, 'F', 'LOW', 'HIGH', 18.043],
       [22, 'F', 'NORMAL', 'HIGH', 8.607],
       [49, 'F', 'NORMAL', 'HIGH', 16.275],
       [41, 'M', 'LOW', 'HIGH', 11.037],
       [60, 'M', 'NORMAL', 'HIGH', 15.171],
       [43, 'M', 'LOW', 'NORMAL', 19.368]], dtype=object)

Một vài trường trong dữ liệu có dạng categorical, như __Sex__ và __BP__. Mô hình Random Forest trong thư viện này chỉ xử lý với dữ liệu số thực do đó chúng ta cần chuyển các thuộc tính trên về dạng số thực tương ứng. Sử dụng  __skearn.preprocessing__ để chuyển dữ liệu dạng categorical về số thực

In [42]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder() #chuẩn hoá nhãn
le_sex.fit(['F','M']) # fit(y): phù hợp với bộ mã hoá nhãn
X[:,1] = le_sex.transform(X[:,1])  # chuyển đổi nhãn thành mã hoá chuẩn hoá 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3])

print(X[0:5])

[[23 0 0 0 25.355]
 [47 1 1 0 13.093]
 [47 1 1 0 10.114]
 [28 0 2 0 7.798]
 [61 0 1 0 18.043]]


Chuẩn hóa thuộc tính __Na_to_K__ về __mean__=0 và __std__=1

In [44]:
mean = X[:, -1].mean() # tính giá trị trung bình 
std = X[:, -1].std() # tính toán độ lệch chuẩn của dữ liệu 
X[:, -1] = (X[:, -1] - mean) / std
print(X[0:5])

[[23 0 0 0 1.2865221173753494]
 [47 1 1 0 -0.4151453955143373]
 [47 1 1 0 -0.8285581765368765]
 [28 0 2 0 -1.1499626749753462]
 [61 0 1 0 0.27179427083731394]]


Dữ liệu target về loại thuốc phản ứng

In [45]:
y = my_data["Drug"]
print(y[0:5])

0    drugY
1    drugC
2    drugC
3    drugX
4    drugY
Name: Drug, dtype: object


Đưa __y__ về dạng số thực

In [46]:
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['drugA','drugB', 'drugC', 'drugX', 'drugY'])
y = le_sex.transform(y) 

In [47]:
print(y[0:5])

[4 2 2 3 4]
